In [16]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.model_selection import train_test_split
from sklearn import metrics
data={
    'age': ['<=30', '<=30', '31...40', '>40', '>40', '>40', '<=30', '<=30', '>40', '31...40'],
    'income': ['high', 'high', 'medium', 'low', 'low', 'low', 'medium', 'low', 'medium', 'medium'],
    'student': ['no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'yes', 'yes'],
    'credit_rating': ['fair', 'fair', 'fair', 'fair', 'fair', 'excellent', 'fair', 'excellent', 'excellent', 'excellent'],
    'buys_computer': ['no', 'no', 'yes', 'no', 'yes', 'no', 'yes', 'no', 'yes', 'yes']
}
data = pd.DataFrame(data)
data

,age,income,student,credit_rating,buys_computer
0,<=30,high,no,fair,no
1,<=30,high,no,fair,no
2,31...40,medium,no,fair,yes
3,>40,low,yes,fair,no
4,>40,low,yes,fair,yes
5,>40,low,yes,excellent,no
6,<=30,medium,no,fair,yes
7,<=30,low,no,excellent,no
8,>40,medium,yes,excellent,yes
9,31...40,medium,yes,excellent,yes


In [10]:
#A1 For the data provided below, calculate the prior probability for each class.
print("Prior probabilties for each features w.r.t 'buys_computer'\n")
for column in data.columns:
    if column != 'buys_computer':
        print(f"\nPrior Probabilities for {column.upper()}:")
        prior_probabilities = data.groupby(column)['buys_computer'].value_counts(normalize=True)
        print(prior_probabilities)
        print()
print("Prior probabilities for the 'buys_computer' classes\n")
total_instances = len(data)
buys_computer_counts = data['buys_computer'].value_counts()
prior_prob_yes = buys_computer_counts['yes'] / total_instances
prior_prob_no = buys_computer_counts['no'] / total_instances
print(f"Prior probability for buys_computer = 'yes': {prior_prob_yes}")
print(f"Prior probability for buys_computer = 'no': {prior_prob_no}")

Prior probabilties for each features w.r.t 'buys_computer'


Prior Probabilities for AGE:
age      buys_computer
31...40  yes              1.00
<=30     no               0.75
         yes              0.25
>40      no               0.50
         yes              0.50
Name: buys_computer, dtype: float64


Prior Probabilities for INCOME:
income  buys_computer
high    no               1.00
low     no               0.75
        yes              0.25
medium  yes              1.00
Name: buys_computer, dtype: float64


Prior Probabilities for STUDENT:
student  buys_computer
no       no               0.6
         yes              0.4
yes      yes              0.6
         no               0.4
Name: buys_computer, dtype: float64


Prior Probabilities for CREDIT_RATING:
credit_rating  buys_computer
excellent      no               0.5
               yes              0.5
fair           no               0.5
               yes              0.5
Name: buys_computer, dtype: float64

Prior probabilities

In [11]:
# A2.Calculate the class conditional densities for various features & classes.
#  Observe if any class conditional density has zero values.
for col in ['age', 'income', 'student', 'credit_rating']:
    print(f"\nClass conditional density for {col}:")
    for label in data['buys_computer'].unique():
        density = data[data['buys_computer'] == label][col].value_counts(normalize=True)
        print(f"Class '{label}':\n{density}\n")


Class conditional density for age:
Class 'no':
<=30    0.6
>40     0.4
Name: age, dtype: float64

Class 'yes':
31...40    0.4
>40        0.4
<=30       0.2
Name: age, dtype: float64


Class conditional density for income:
Class 'no':
low     0.6
high    0.4
Name: income, dtype: float64

Class 'yes':
medium    0.8
low       0.2
Name: income, dtype: float64


Class conditional density for student:
Class 'no':
no     0.6
yes    0.4
Name: student, dtype: float64

Class 'yes':
yes    0.6
no     0.4
Name: student, dtype: float64


Class conditional density for credit_rating:
Class 'no':
fair         0.6
excellent    0.4
Name: credit_rating, dtype: float64

Class 'yes':
fair         0.6
excellent    0.4
Name: credit_rating, dtype: float64



In [12]:
# A3.Test for independence between the 4 given features.
import pandas as pd
from scipy.stats import chi2_contingency
for feature in data.columns[:-1]:
    for target_feature in data.columns[:-1]:
        if feature != target_feature:
            contingency_table = pd.crosstab(data[feature], data[target_feature])
            chi2, p, _, _ = chi2_contingency(contingency_table)

            alpha = 0.05
            print(f"\nChi-square test for independence between '{feature}' and '{target_feature}':")
            print(f"p-value: {p}")
            if p < alpha:
                print("dependent.")
            else:
                print("independent.")


Chi-square test for independence between 'age' and 'income':
p-value: 0.11170929281604328
independent.

Chi-square test for independence between 'age' and 'student':
p-value: 0.018315638888734182
dependent.

Chi-square test for independence between 'age' and 'credit_rating':
p-value: 0.7316156289466418
independent.

Chi-square test for independence between 'income' and 'age':
p-value: 0.1117092928160433
independent.

Chi-square test for independence between 'income' and 'student':
p-value: 0.22313016014842982
independent.

Chi-square test for independence between 'income' and 'credit_rating':
p-value: 0.4345982085070783
independent.

Chi-square test for independence between 'student' and 'age':
p-value: 0.018315638888734182
dependent.

Chi-square test for independence between 'student' and 'income':
p-value: 0.22313016014842982
independent.

Chi-square test for independence between 'student' and 'credit_rating':
p-value: 0.5186050164287255
independent.

Chi-square test for independenc

In [14]:
# A4. Build a Naïve-Bayes (NB) classifier for the above given data.
label_encoder = LE()
data_encoded = data.copy()
for col in data.columns:
    if data[col].dtype == 'object':
        data_encoded[col] = label_encoder.fit_transform(data[col])
X = data_encoded.loc[:, 'age':'credit_rating']
y = data_encoded['buys_computer']
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2, random_state=42)
model = GaussianNB()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(X_test)
print("\nPredicted classes for test data:")
print(predicted)

   age  income  student  credit_rating
8    2       2        1              0
1    1       0        0              1

Predicted classes for test data:
[1 0]


In [39]:
# A5. Build a Naïve-Bayes (NB) classifier for your dataset.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('train.csv')
label_encoder = LabelEncoder()
df['Crop_Type'] = label_encoder.fit_transform(df['Crop_Type'])
df['Soil_Type'] = label_encoder.fit_transform(df['Soil_Type'])
df['Pesticide_Use_Category'] = label_encoder.fit_transform(df['Pesticide_Use_Category'])
df['Season'] = label_encoder.fit_transform(df['Season'])
df = df.fillna(df.mean())
X = df.drop(['ID', 'Crop_Damage'], axis=1)
y = df['Crop_Damage']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Accuracy: 0.8166779203241054


<ipython-input-39-9d4d87f9a8eb>:12: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.mean())
